## Agregar las variables de entorno

In [5]:
import os

# Parte de donde sea que esté el CWD actualmente
d = os.getcwd()

# Subir hasta encontrar la carpeta "Entregable"
while os.path.basename(d) != "Entregable":
    parent = os.path.dirname(d)
    if parent == d:   # Llegó al disco sin encontrar Entregable
        raise RuntimeError("No se encontró la carpeta Entregable hacia arriba.")
    d = parent

os.chdir(d)
print("CWD fijado en:", os.getcwd())

CWD fijado en: c:\Users\Mariana\Dropbox\Cursos\SoyHenry\DataEngineering\Modulo02-ModeladoDeDatosYBDRelacionales\ProyectoIntegrador\Entregable


In [6]:
# --- Configuración del entorno ---
import os
import sys
from dotenv import load_dotenv

load_dotenv()

# Variables de entorno (ya definidas en .env)
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_DB = os.getenv("PG_DB")

print(f"Conectando a PostgreSQL en {PG_HOST}:{PG_PORT}, base {PG_DB}")


Conectando a PostgreSQL en localhost:5432, base ecommerce


## Crear conexión y Engine

In [7]:
from src.database.connection import DB

# Crea la base si no existe y devuelve engine/session
engine = DB.engine()
SessionLocal = DB.session()

print("Engine y sesión inicializados")


Engine y sesión inicializados


In [8]:
from src.database.connection import DB

engine = DB.engine()

try:
    conn = engine.connect()
    print("¡Conexión OK!", conn.execute("SELECT 1").fetchone())
except Exception as e:
    print("ERROR:", e.__class__.__name__, str(e))

ERROR: ObjectNotExecutableError Not an executable object: 'SELECT 1'


In [10]:
# Borrar las tablas creadas con anterioridad

from src.database.connection import DB
from src.database.models import Base

engine = DB.engine()

Base.metadata.drop_all(bind=engine)
print("Todas las tablas eliminadas")

Todas las tablas eliminadas


## Crear las tablas ORM

In [11]:
from src.database.models import Base
Base.metadata.create_all(engine)
print("Tablas creadas (si no existían)")


Tablas creadas (si no existían)


## Cargar los .csv con loader

In [12]:
from src.services.loader import load_all

load_all()
print("Datos cargados desde CSVs")


Datos cargados desde CSVs


## Verificar cantidad en cada tabla

In [13]:
from sqlalchemy import text
schema = "staging"

with engine.connect() as conn:
    for tabla in ["usuarios", "categorias", "productos", "ordenes", "detalle_ordenes"]:
        count = conn.execute(text(f"SELECT COUNT(*) FROM {schema}.{tabla}")).scalar()
        print(f"{tabla}: {count}")


usuarios: 1000
categorias: 12
productos: 36
ordenes: 10000
detalle_ordenes: 10000


## Consultas exploratorias

In [14]:
import pandas as pd
from sqlalchemy import text

q = """
SELECT c.nombre AS categoria, p.nombre AS producto, SUM(d.cantidad) AS unidades
FROM staging.detalle_ordenes d
JOIN staging.productos p ON p.id = d.producto_id
JOIN staging.categorias c ON c.id = p.categoria_id
GROUP BY c.nombre, p.nombre
ORDER BY c.nombre, unidades DESC
LIMIT 10;
"""

df = pd.read_sql(text(q), engine)
df


,categoria,producto,unidades
0,Automotriz,Aceite de Motor 5W30,844
1,Automotriz,Cámara de Reversa,777
2,Automotriz,Limpiaparabrisas Universal,777
3,Belleza y Cuidado Personal,Crema Hidratante Facial,818
4,Belleza y Cuidado Personal,Perfume Hombre 100ml,770
5,Belleza y Cuidado Personal,Plancha para Cabello Remington,707
6,Deportes y Aire Libre,Mochila Deportiva Nike,771
7,Deportes y Aire Libre,Bicicleta Montaña Aro 29,757
8,Deportes y Aire Libre,Pelota de Fútbol Profesional,750
9,Electrónica,Laptop Dell Inspiron 15,847
